In [1]:
import torch
from PIL import Image
from torchvision import transforms, models
import torch.nn as nn

In [2]:
def classificate_image(image_path, model, label_encoder, device):
    image = Image.open(image_path)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])
    image_tensor = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        model.eval()
        output = model(image_tensor)
    _, predicted_idx = torch.max(output, 1)
    
    predicted_idx = predicted_idx.item()
    predicted_label = label_encoder.get_label(predicted_idx)

    return predicted_label


In [5]:
class Label_encoder:
    def __init__(self, labels):
        labels = list(set(labels))
        self.labels = {label: idx for idx, label in enumerate(classes)}

    def get_label(self, idx):
        return list(self.labels.keys())[idx]

    def get_idx(self, label):
        return self.labels[label]
classes = open("./food-101/meta/classes.txt", 'r').read().splitlines()
encoder = Label_encoder(classes)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = models.efficientnet_v2_s(weights=None)

model.load_state_dict(torch.load('effnet_fin.pth'))
model.to(device)
model.eval()

label_encoder = Label_encoder(classes)



In [8]:
image_path = "food-101/images/risotto/36631.jpg" 
predicted_label = classificate_image(image_path, model, label_encoder, device)
print("Predicted Label:", predicted_label)

Predicted Label: risotto
